In [1]:
exec(open("../../TrainingAlphas/Transformer/Transformer.py").read())

import argparse
import json

import h5py
from captum.attr import IntegratedGradients
from torch.utils.data import DataLoader, Dataset


class InferenceDataset(Dataset):
    def __init__(self, file):
        self.filename = file
        f = h5py.File(file, "r")
        self.length = f["anime"].shape[0]
        self.embeddings = [
            f["anime"][:] - 1,
            f["manga"][:] - 1,
            f["rating"][:].reshape(*f["rating"].shape, 1).astype(np.float32),
            f["timestamp"][:].reshape(*f["timestamp"].shape, 1).astype(np.float32),
            f["status"][:] - 1,
            f["completion"][:].reshape(*f["completion"].shape, 1).astype(np.float32),
            f["position"][:] - 1,
        ]
        self.mask = f["user"][:]

        def process_position(x):
            return x[:].flatten().astype(np.int64) - 1

        self.positions = process_position(f["positions"])

    def __len__(self):
        return self.length

    def __getitem__(self, i):
        embeds = tuple(x[i, :] for x in self.embeddings)
        mask = self.mask[i, :]
        mask = mask.reshape(1, mask.size) != mask.reshape(mask.size, 1)
        positions = self.positions[i]
        return embeds, mask, positions


def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device)


def get_model(medium, task):
    device = torch.device("cuda")
    source_dir = get_data_path(
        os.path.join("alphas", medium, task, "Transformer", "v1")
    )
    model_file = os.path.join(source_dir, "model.pt")
    config_file = os.path.join(source_dir, "config.json")
    training_config = create_training_config(config_file, 1)
    model_config = create_model_config(training_config)
    model = TransformerModel(model_config)
    model.load_state_dict(load_model(model_file, map_location="cpu"))
    model = model.to(device)
    model.eval()
    return model


def get_data(username, medium, task):
    device = torch.device("cuda")
    outdir = get_data_path(
        f"recommendations/{username}/alphas/{medium}/{task}/Transformer/v1"
    )
    dataloader = DataLoader(
        InferenceDataset(os.path.join(outdir, "inference.h5")),
        batch_size=1,
        shuffle=False,
    )
    it = iter(dataloader)
    data = tuple(next(it))
    return to_device(data, device)


def get_baseline(medium, task, inputs, positions, fields):
    # load configs
    source_dir = get_data_path(
        os.path.join("alphas", medium, task, "Transformer", "v1")
    )
    config_file = os.path.join(source_dir, "config.json")
    config = json.load(open(config_file, "r"))
    training_config = create_training_config(config_file, 1)

    # get mask tokens
    assert len(training_config["vocab_types"]) == 8
    assert training_config["vocab_types"][6] == None
    mask_tokens = config["mask_tokens"][:6] + config["mask_tokens"][7:]
    empty_tokens = config["empty_tokens"][:6] + config["mask_tokens"][7:]    
    vocab_types = (
        training_config["vocab_types"][:6] + training_config["vocab_types"][7:]
    )
    for i in range(len(vocab_types)):
        mask_tokens[i] -= vocab_types[i] == int
        empty_tokens[i] -= vocab_types[i] == int

    # # mask out item inputs
    # baseline is a new user with no watched items
    end_of_seq_pos = positions.cpu().numpy()[0]
    baseline_inputs = [x.clone() for x in inputs]
    for i in fields:
        for j in range(1, end_of_seq_pos):
            if baseline_inputs[0][0, j] != empty_tokens[i]:
                baseline_inputs[i][0, j] = np.random.randint(mask_tokens[i])
                # baseline_inputs[i][0, j] = mask_tokens[i]
    return baseline_inputs
    

def cpu(x):
    return x.detach().cpu().numpy().squeeze().tolist()


def utility_model(
    anime_embedding,
    manga_embedding,
    rating_embedding,
    timestamp_embedding,
    status_embedding,
    completion_embedding,
    position_embedding,
    model=None,
    positions=None,
    mask=None,
    coefs=None,
    medium=None,
):
    embedding = (
        anime_embedding
        + manga_embedding
        + rating_embedding
        + timestamp_embedding
        + status_embedding
        + completion_embedding
        + position_embedding
    )
    embedding = model.embed.postprocessor(embedding)
    hidden = model.transformers(embedding, mask)
    output = hidden[range(len(positions)), positions, :]
    if medium == "anime":
        idxp = 0
        idxr = 1
    elif medium == "manga":
        idxp = 2
        idxr = 3
    else:
        assert False
    p = model.classifier[idxp](output)
    r = model.classifier[idxr](output)
    # transform into MLE utility
    p = torch.exp(p)
    r = torch.relu((r + 10) / 10)
    u = (
        np.exp(coefs[0]) * p
        + np.exp(coefs[1]) * r
        + np.exp(coefs[2]) * torch.log(p)
        + np.exp(coefs[3]) * torch.log(r)
    )
    return u

def zero_out(x, pos):
    x[:, 1:pos, :] = 0
    return x

def identity(x, pos):
    return x


def compute_attributions(username, medium, task, coefs, items, fields):
    model = get_model(medium, task)
    inputs, mask, positions = get_data(username, medium, task)
    baseline_inputs = get_baseline(medium, task, inputs, positions, fields)

    embedding = tuple(embed(x) for (embed, x) in zip(model.embed.embeddings, inputs))
    baseline_embedding = tuple(
        embed(x) for (embed, x) in zip(model.embed.embeddings, baseline_inputs)
    )

    ig = IntegratedGradients(utility_model)
    attributions = {}
    for item in items:
        attrs = ig.attribute(
            embedding,
            baseline_embedding,
            target=item,
            internal_batch_size=8,
            additional_forward_args=(model, positions, mask, coefs, medium),
            n_steps=20,
        )
        return cpu(sum(attrs[:2]).sum(dim=2))        
        # return cpu(sum(attrs).sum(dim=2))
        attrs = sum(attrs)
        attributions[item] = cpu(attrs.sum(dim=2))
    return attributions, cpu(inputs[0]), cpu(inputs[1])


def save_attributions(username, medium, task, coefs, items):
    cache = {}
    attrs, anime, manga = compute_attributions(username, medium, task, coefs, items)
    cache["anime"] = anime
    cache["manga"] = manga
    cache["attributions"] = attrs
    return cache

SyntaxError: invalid syntax (2092076523.py, line 1)

In [2]:
username = "Fro116"
medium = "anime"
task = "temporal_causal"
coefs = [-5.481506, 1.2974571, -1.084365, 0.24383727]
# save_attributions(username, medium, task, coefs, items)

In [3]:
inputs, mask, positions = get_data(username, medium, task)

In [4]:
inputs[0][0, 585]

tensor(6661, device='cuda:0')

In [5]:
bocchi = 6437
kanojo = 22311
ojamajo = 5802
bunny = 19765
tengoku = 5667
fullmoon = 8706
touch = 23762
items = [touch]

In [6]:
# bunny -> bunny
# kanojo -> kanojo
# full moon -> kodocha (boo!)
# bocchi -> akb0048 
# ojamajo -> aria
# touch -> kodocha (boo!)

In [7]:
# anime_attrs = compute_attributions(username, medium, task, coefs, items, [0, 1, 2, 3, 4, 5, 6])
# rating_attrs = compute_attributions(username, medium, task, coefs, items, [2])
# ts_attrs = compute_attributions(username, medium, task, coefs, items, [3])
# pos_attrs = compute_attributions(username, medium, task, coefs, items, [1, 3])

In [8]:
from tqdm import tqdm

In [9]:
attrs = []
for i in tqdm(range(30)):
    attrs.append(compute_attributions(username, medium, task, coefs, items, [0, 1, 2, 3, 4, 5]))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:46<00:00,  1.56s/it]


In [10]:
avg_attrs = sum(np.array(x) for x in attrs)
[inputs[0][0, np.argsort(avg_attrs)[-1-x]] for x in range(5)]

[tensor(821, device='cuda:0'),
 tensor(24957, device='cuda:0'),
 tensor(11349, device='cuda:0'),
 tensor(11961, device='cuda:0'),
 tensor(6040, device='cuda:0')]

In [11]:
[inputs[1][0, np.argsort(avg_attrs)[-1-x]] for x in range(5)]

[tensor(66823, device='cuda:0'),
 tensor(66823, device='cuda:0'),
 tensor(66823, device='cuda:0'),
 tensor(66823, device='cuda:0'),
 tensor(66823, device='cuda:0')]

In [12]:
np.sort(attrs)

array([[-0.05014963, -0.04879337, -0.04778021, ...,  0.05706015,
         0.06754141,  0.10395877],
       [-0.16695583, -0.09790334, -0.06586245, ...,  0.04741236,
         0.05375742,  0.14209028],
       [-0.17932507, -0.08462202, -0.05074595, ...,  0.09995314,
         0.15306959,  0.26410297],
       ...,
       [-0.14195417, -0.08618962, -0.0652843 , ...,  0.07789235,
         0.08925848,  0.16869851],
       [-0.04940343, -0.04839007, -0.03370804, ...,  0.06859194,
         0.08550637,  0.12960392],
       [-0.14423118, -0.07191206, -0.06692154, ...,  0.06403629,
         0.06978307,  0.09930972]])